In [1]:
# 01_data_extraction.ipynb

# -----------------------------
# 🔹 1. Importar librerías
# -----------------------------
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt

# -----------------------------
# 🔹 2. Cargar archivo Parquet
# -----------------------------
DATA_DIR = Path("../data/raw")
file_path = list(DATA_DIR.glob("weekly_*.parquet"))[0]  # Carga el primer archivo disponible

print(f"Cargando archivo: {file_path}")
df = pl.read_parquet(file_path)

# Mostrar estructura
print("Dimensiones:", df.shape)
print("Columnas disponibles:", df.columns[:10], "...")
df.head(5)


IndexError: list index out of range

In [ ]:
# -----------------------------
# 🔹 3. Información general
# -----------------------------
print(df.describe())

# Ver equipos únicos
print("\nEquipos únicos (field_team):")
print(df["field_team"].unique().to_list())

# Ver jugadores únicos
print("\nNúmero de jugadores registrados:")
print(df["player_id"].n_unique())


In [ ]:
# -----------------------------
# 🔹 4. Ejemplo: estadísticas de un jugador
# -----------------------------
# Filtrar un jugador al azar
sample_player = df.select("player_display_name").sample(1).to_series()[0]
print(f"Mostrando registros del jugador: {sample_player}")

player_df = df.filter(pl.col("player_display_name") == sample_player)
player_df.select([
    "season", "week", "player_display_name", "recent_team",
    "position", "passing_yards", "rushing_yards", "receiving_yards"
]).head(10)


In [ ]:
# -----------------------------
# 🔹 5. Visualización rápida
# -----------------------------
# Ejemplo: Yardas aéreas promedio por semana

yardas = (
    df.group_by("week")
    .agg(pl.col("passing_yards").mean().alias("avg_passing_yards"))
    .sort("week")
)

plt.figure(figsize=(10,5))
plt.plot(yardas["week"], yardas["avg_passing_yards"], marker="o")
plt.title("Promedio de yardas de pase por semana")
plt.xlabel("Semana")
plt.ylabel("Yardas promedio")
plt.grid(True)
plt.show()
